In [ ]:
import pandas as pd
import numpy as np
import random

class Problema:
    def __init__(self, arquivo_distancia, arquivo_tempo):
        self.distancia = pd.read_csv(arquivo_distancia, header=None).to_numpy(dtype=float)
        self.tempo     = pd.read_csv(arquivo_tempo,     header=None).to_numpy(dtype=float)
        self.n = self.distancia.shape[0]  # número de cidades

    def custo_arco(self, i, j, objetivo):
        # objetivo: "distancia" ou "tempo"
        if objetivo == "distancia":
            return self.distancia[i, j]
        else:
            return self.tempo[i, j]

def custo_rota(problema, rota, objetivo):
    # Soma o custo ao longo da rota circular (0 -> ... -> 0)
    c = 0.0
    for k in range(len(rota) - 1):
        c += problema.custo_arco(rota[k], rota[k+1], objetivo)
    c += problema.custo_arco(rota[-1], rota[0], objetivo)  # volta à origem
    return c

def construtiva_vizinho_mais_proximo(problema, objetivo):
    # Gera uma rota inicial simples: vizinho mais próximo a partir da cidade 0.
    n = problema.n
    rota = [0]
    visitado = [False] * n
    visitado[0] = True
    atual = 0

    for cidade in range(n - 1):  # repetimos n-1 escolhas 
        proxima_cidade = None
        menor_custo = float("inf")

        for j in range(n):
            if not visitado[j]:
                custo = problema.custo_arco(atual, j, objetivo)
                if custo < menor_custo:
                    menor_custo = custo
                    proxima_cidade = j

        rota.append(proxima_cidade)
        visitado[proxima_cidade] = True
        atual = proxima_cidade

    return rota

def main():
    problema = Problema("distancia.csv", "tempo.csv")

    # escolha qual função objetivo otimizar nesta execução
    objetivo = "distancia"  # "distancia" ou "tempo"

    # rota inicial (vizinho mais próximo)
    rota = construtiva_vizinho_mais_proximo(problema, objetivo)

    # custo da rota
    c = custo_rota(problema, rota, objetivo)

    print(f"n = {problema.n}")
    print(f"Objetivo: {objetivo}")
    print(f"Custo da rota inicial: {c:.3f}")
    print(f"Rota (primeiros 10 nós): {rota[:10]}")

main()


n = 250
Objetivo: distancia
Custo da rota inicial: 1586.700
Rota (primeiros 10 nós): [0, 72, 214, 248, 33, 120, 22, 11, 169, 51]


In [9]:
# definição das vizinhanças 

# K1: SWAP (2-exchange) — troca as posições a e b
def vizinhanca_swap(rota, a, b):
    # não mexe na posição 0 e ignora casos triviais
    if a == 0 or b == 0 or a == b:
        return rota
    n = len(rota)
    if not (0 <= a < n and 0 <= b < n):
        return rota
    nova = rota.copy()
    nova[a], nova[b] = nova[b], nova[a]
    return nova

# K2: OR-OPT (k=1) — remove a cidade em a e reinsere antes da posição b
def vizinhanca_oropt1(rota, a, b):
    # não mexe na posição 0; evita b == a ou b == a+1 (inserção nula)
    if a == 0 or b == 0 or b == a or b == a + 1:
        return rota
    n = len(rota)
    if not (0 <= a < n and 0 <= b <= n):  # b pode ser n (inserir no fim antes de voltar ao 0)
        return rota
    nova = rota.copy()
    cidade = nova.pop(a)
    if b > a:
        b -= 1  # corrige deslocamento do índice após o pop
    nova.insert(b, cidade)
    return nova

# K3: 2-OPT (dirigido) — reverte o segmento [a, b]
def vizinhanca_2opt(rota, a, b):
    # mantém a origem na posição 0; exige 1 <= a < b <= n-2 para preservar a ligação final com 0
    n = len(rota)
    if a <= 0 or b >= n - 1 or a >= b:
        return rota
    nova = rota.copy()
    nova[a:b+1] = reversed(nova[a:b+1])
    return nova

# aplica a vizinhança K em (a, b)
# K=1 -> swap, K=2 -> or-opt(1), K=3 -> 2-opt
def aplicar_vizinhanca(rota, k, a, b):
    if k == 1:
        return vizinhanca_swap(rota, a, b)
    if k == 2:
        return vizinhanca_oropt1(rota, a, b)
    if k == 3:
        return vizinhanca_2opt(rota, a, b)

    return rota


In [ ]:
# busca local

def diferenca_custo_k1(problema, rota, a, b, objetivo, M=None):
    # K1: swap a<b 
    n = len(rota)
    if a == 0 or b == 0 or a >= b:
        return 0.0
    if M is None:
        M = problema.distancia if objetivo == "distancia" else problema.tempo

    A_prev = rota[a-1]; A = rota[a]; A_next = rota[a+1] if a+1 < n else rota[0]
    B_prev = rota[b-1]; B = rota[b]; B_next = rota[b+1] if b+1 < n else rota[0]

    antigo = M[A_prev, A] + M[A, A_next] + M[B_prev, B] + M[B, B_next]

    if b == a + 1:
        novo = M[A_prev, B] + M[B, A] + M[A, B_next]
    else:
        novo = M[A_prev, B] + M[B, A_next] + M[B_prev, A] + M[A, B_next]

    return novo - antigo

def diferenca_custo_k2(problema, rota, a, b, objetivo, M=None):
    # K2: or-opt(1) remove em a e reinsere antes de b
    n = len(rota)
    if a == 0 or b == 0 or b == a or b == a + 1:
        return 0.0
    if M is None:
        M = problema.distancia if objetivo == "distancia" else problema.tempo

    pred_a = rota[a-1]; a_i = rota[a]; suc_a = rota[a+1] if a+1 < n else rota[0]
    pred_b = rota[b-1]; b_i = rota[b] if b < n else rota[0]

    antigo = M[pred_a, a_i] + M[a_i, suc_a] + M[pred_b, b_i]
    novo   = M[pred_a, suc_a] + M[pred_b, a_i] + M[a_i, b_i]

    return novo - antigo

def diferenca_custo_k3(problema, rota, a, b, objetivo, M=None):
    # K3: 2-opt reverte [a,b]
    n = len(rota)
    if a <= 0 or b >= n - 1 or a >= b:
        return 0.0
    if M is None:
        M = problema.distancia if objetivo == "distancia" else problema.tempo

    i = rota[a-1]; j = rota[a]; k = rota[b]; l = rota[b+1]
    antigo = M[i, j] + M[k, l]
    novo   = M[i, k] + M[j, l]
    return novo - antigo


def first_improvement(problema, rota, objetivo, k):
     # retorna (rota_final, custo_final) após atingir ótimo local em N_k
    n = len(rota)
    rota_atual = rota[:]
    M = problema.distancia if objetivo == "distancia" else problema.tempo

    # custo atual calculado direto na matriz M
    custo_atual = 0.0
    for t in range(n-1):
        custo_atual += M[rota_atual[t], rota_atual[t+1]]
    custo_atual += M[rota_atual[-1], rota_atual[0]]

    while True:
        melhorou = False

        if k == 1:
            # K1: SWAP — varre pares (a,b) na ordem e para na primeira melhora
            for a in range(1, n-1):
                if melhorou: break
                for b in range(a+1, n):
                    d = diferenca_custo_k1(problema, rota_atual, a, b, objetivo, M=M)
                    if d < 0.0:
                        rota_atual = vizinhanca_swap(rota_atual, a, b)
                        custo_atual += d
                        melhorou = True
                        break

        elif k == 2:
            # K2: OR-OPT(1) — remove em a e insere antes de b
            for a in range(1, n-1):
                if melhorou: break
                for b in range(1, n):
                    if b == a or b == a + 1:
                        continue
                    d = diferenca_custo_k2(problema, rota_atual, a, b, objetivo, M=M)
                    if d < 0.0:
                        rota_atual = vizinhanca_oropt1(rota_atual, a, b)
                        custo_atual += d
                        melhorou = True
                        break

        elif k == 3:
            # K3: 2-OPT — reverte [a,b]
            for a in range(1, n-2):
                if melhorou: break
                for b in range(a+1, n-1):
                    d = diferenca_custo_k3(problema, rota_atual, a, b, objetivo, M=M)
                    if d < 0.0:
                        rota_atual = vizinhanca_2opt(rota_atual, a, b)
                        custo_atual += d
                        melhorou = True
                        break

        # se uma varredura completa não encontrou melhora, atingiu ótimo local em N_k
        if not melhorou:
            break

    return rota_atual, custo_atual

def neighborhood_change(rota, custo, rota_cand, custo_cand, k):
    if custo_cand < custo:
        return rota_cand, custo_cand, 1
    else:
        return rota, custo, k + 1

# TO DO: o professor falou que ao inves de voltar sempre para k=1 seria interessante randomizar as vizinhanças, vou fazer isso com o metodo rnvd mas preciso testar
def vnd(problema, rota_inicial, objetivo, kmax=3):
    rota = rota_inicial[:]
    custo = custo_rota(problema, rota, objetivo)
    k = 1
    while k <= kmax:
        rota_cand, custo_cand = first_improvement(problema, rota, objetivo, k)
        rota, custo, k = neighborhood_change(rota, custo, rota_cand, custo_cand, k)
    return rota, custo


In [ ]:
def rvnd(problema, rota_inicial, objetivo, seed=None):
    if seed is not None:
        random.seed(seed)  # usa estado global
    rota = rota_inicial[:]
    custo = custo_rota(problema, rota, objetivo)
    Ks = [1, 2, 3]  # ou [3,2,1] se quiser começar por 2-opt com mais frequência

    while True:
        lista = Ks[:]
        random.shuffle(lista)  # embaralha ordem das vizinhanças
        melhorou = False

        for k in lista:
            # use a versão "pura" do FI se quer ótimo local dentro de N_k
            rota_cand, custo_cand = first_improvement(problema, rota, objetivo, k)
            if custo_cand < custo:
                rota, custo = rota_cand, custo_cand
                melhorou = True
                break  # melhora aceita -> reembaralha e recomeça

        if not melhorou:
            break  # nenhuma K melhorou -> ótimo local em RVND

    return rota, custo


In [ ]:
def shake(rota, k):
    n = len(rota)

    if k == 1:
        # SWAP: 1 <= a < b <= n-1
        a = random.randrange(1, n-1)
        b = random.randrange(a+1, n)
        return vizinhanca_swap(rota, a, b)

    elif k == 2:
        # OR-OPT(1): remove em a e reinsere antes de b
        # a em [1, n-2]; b em [1, n] e b ≠ a, b ≠ a+1
        a = random.randint(1, n-2)
        b = random.randint(1, n)  # pode ser n (inserir no fim)
        while b == a or b == a + 1:
            b = random.randint(1, n)
        return vizinhanca_oropt1(rota, a, b)

    elif k == 3:
        # 2-OPT: 1 <= a < b <= n-2
        a = random.randrange(1, n-2)
        b = random.randrange(a+1, n-1)
        return vizinhanca_2opt(rota, a, b)

    return rota[:]  # k inválido


In [17]:
def gvns(problema, rota_inicial, objetivo, kmax=3, max_iter=1000):
    # x = solução corrente; fx = custo corrente
    x = rota_inicial[:]
    fx = custo_rota(problema, x, objetivo)

    it = 0
    while it < max_iter:
        k = 1
        melhorou_na_escada = False

        # percorre k = 1..kmax (escada de vizinhanças)
        while k <= kmax and it < max_iter:
            # 1) shaking
            x_pr = shake(x, k)

            # 2) busca local via RVND
            x_b, f_b = rvnd(problema, x_pr, objetivo)

            # 3) neighborhood change
            x, fx, k = neighborhood_change(x, fx, x_b, f_b, k)

            if k == 1 and f_b < fx:
                melhorou_na_escada = True

            it += 1

        # se subiu a escada inteira sem melhorar, encerra
        if not melhorou_na_escada:
            break

    return x, fx


In [ ]:
# comparação: rota inicial (NN) vs. GVNS

# instancia o problema
problema = Problema("distancia.csv", "tempo.csv")

# define qual objetivo otimizar: "distancia" ou "tempo"
objetivo = "distancia"

# rota inicial por Vizinho Mais Próximo
rota0 = construtiva_vizinho_mais_proximo(problema, objetivo)
c0 = custo_rota(problema, rota0, objetivo)

# roda o GVNS
kmax = 3        # temos 3 vizinhanças (swap, or-opt(1), 2-opt)
max_iter = 2000 # limite de iterações externas
rota_g, c_g = gvns(problema, rota0, objetivo, kmax=kmax, max_iter=max_iter)

# imprime resultados
print(f"n = {problema.n}")
print(f"Objetivo: {objetivo}")
print(f"Custo rota inicial (NN): {c0:.3f}")
print(f"Custo após GVNS:        {c_g:.3f}")

n = 250
Objetivo: distancia
Custo rota inicial (NN): 1586.700
